In [1]:
import pandas as pd
import numpy as np
import time

In [2]:
df = pd.read_excel('raw_data_with_full_label.xlsx', sheet_name='links', index_col=0).dropna(subset=['y'])
df.head()

,link,title,canal_nome,canal_link,view_counts,video_date,like,dislike,y
0,https://www.youtube.com/watch?v=-0WQnwNFqJM,AI learns to Speedrun QWOP using Machine Learning,Wesley Liao,https://www.youtube.com/channel/UCair-JZ6XmtJk...,226613.0,25 de fev. de 2021,"2.729 marcações ""Gostei""","33 marcações ""Não gostei""",0.0
1,https://www.youtube.com/watch?v=-5hEYRt8JE0,How to Become A Machine Learning Engineer | Ho...,#MachineLearningAlgorithms,https://www.youtube.com/hashtag/machinelearnin...,53263.0,3 de set. de 2018,"1.407 marcações ""Gostei""","27 marcações ""Não gostei""",0.0
2,https://www.youtube.com/watch?v=-6RqxhNO2yY,Python For Data Science Full Course - 9 Hours ...,#edureka,https://www.youtube.com/hashtag/edureka,142715.0,15 de mar. de 2020,"3.990 marcações ""Gostei""","57 marcações ""Não gostei""",0.0
3,https://www.youtube.com/watch?v=-AkBfBWr_Gw,Can You Become a Data Scientist?,#data,https://www.youtube.com/hashtag/data,776910.0,14 de ago. de 2018,"12.316 marcações ""Gostei""","431 marcações ""Não gostei""",0.0
4,https://www.youtube.com/watch?v=-DDrE6T0ct4,Binary Image Classification with CNN (Deep Lea...,#download,https://www.youtube.com/hashtag/download,2290.0,23 de mar. de 2020,"66 marcações ""Gostei""","Sem marcações ""Não gostei""",1.0


In [3]:
df.dropna(inplace=True)

In [4]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1452 entries, 0 to 1766
Data columns (total 9 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   link         1452 non-null   object 
 1   title        1452 non-null   object 
 2   canal_nome   1452 non-null   object 
 3   canal_link   1452 non-null   object 
 4   view_counts  1452 non-null   float64
 5   video_date   1452 non-null   object 
 6   like         1452 non-null   object 
 7   dislike      1452 non-null   object 
 8   y            1452 non-null   float64
dtypes: float64(2), object(7)
memory usage: 113.4+ KB


In [5]:
df_limpo = pd.DataFrame(index=df.index)
df_limpo['title'] = df['title']
df_limpo['views'] = df['view_counts']

In [6]:
clean_date = df['video_date'].str.extract(r'(\d+) de (\w+)\. de (\d+)')
clean_date[0] = clean_date[0].map(lambda x: "0"+ x[0] if len(x[0]) == 1 else x)

mapa_meses = {
    'jan': 'Jan',
    'fev': 'Feb',
    'mar': 'Mar',
    'abr': 'Apr',
    'mai': 'May',
    'jun': 'Jun',
    'jul': 'Jul',
    'ago': 'Aug',
    'set': 'Sep',
    'out': 'Oct',
    'nov': 'Nov',
    'dez': 'Dec'
}

clean_date[1] = clean_date[1].map(mapa_meses)
clean_date = clean_date.apply(lambda x: ' '.join(x), axis=1)
clean_date.head()
df_limpo['date'] = pd.to_datetime(clean_date, format='%d %b %Y')

In [7]:
indexs = df_limpo['date'].loc[df_limpo['date'].dt.year < 2018].index.values

df_limpo.drop(index=indexs, axis='rows', inplace=True)
df.drop(index=indexs, axis='rows', inplace=True)

In [8]:
df_limpo.shape, df.shape

((1307, 3), (1307, 9))

In [9]:
features = pd.DataFrame(index=df_limpo.index)
y = df['y'].copy()

features['tempo_desde_pub'] = (pd.to_datetime('2021-03-18') - df_limpo['date']) / np.timedelta64(1, 'D')
features['views'] = df_limpo['views']
features['views_por_dia'] = features['views'] / features['tempo_desde_pub']
features.drop(columns=['tempo_desde_pub'], inplace=True)

In [10]:
qtd = len(df_limpo['date'])
meio = qtd // 2 if qtd % 2 == 0 else (qtd // 2) + 1
date = str(pd.Timestamp(df_limpo.sort_values(by='date')['date'].values[meio]).date())
date

'2020-03-02'

In [11]:
mask_train = (df_limpo['date'] < date)
mask_val = (df_limpo['date'] >= date)

X_train, X_val = features[mask_train], features[mask_val]
y_train, y_val = y[mask_train], y[mask_val]
X_train.shape, X_val.shape, y_train.shape, y_val.shape

((645, 2), (662, 2), (645,), (662,))

In [12]:
from sklearn.feature_extraction.text import TfidfVectorizer

title_train = df_limpo[mask_train]['title']
title_val = df_limpo[mask_val]['title']

title_vec = TfidfVectorizer(min_df=2, ngram_range=(1,4))
title_bow_train = title_vec.fit_transform(title_train)
title_bow_val = title_vec.transform(title_val)

In [13]:
title_bow_train.shape, title_bow_val.shape

((645, 1548), (662, 1548))

In [14]:
from scipy.sparse import hstack

Xtrain_wtitle = hstack([X_train, title_bow_train])
Xval_wtitle = hstack([X_val, title_bow_val])

In [15]:
Xtrain_wtitle.shape, Xval_wtitle.shape

((645, 1550), (662, 1550))

In [16]:
from sklearn.ensemble import RandomForestClassifier

mdl = RandomForestClassifier(n_estimators=1000, random_state=0,min_samples_leaf=1, class_weight='balanced', n_jobs=6)
mdl.fit(Xtrain_wtitle, y_train)

RandomForestClassifier(class_weight='balanced', n_estimators=1000, n_jobs=6,
                       random_state=0)

In [17]:
p = mdl.predict_proba(Xval_wtitle)[:,1]

In [18]:
from sklearn.metrics import roc_auc_score, average_precision_score

In [19]:
average_precision_score(y_val, p)

0.2701322613168307

In [20]:
roc_auc_score(y_val, p)

0.7720751633986929

app 0.24665776077736257 auc 0.7577614379084967 - min_samples_leaf=1 n_estimator=1000 mind_df=1

ap 0.27324126525332704 auc 0.75640522875817 - min_samples_leaf=1 n_estimator=1000 min_df=2

ap 0.2701322613168307 auc 0.7720751633986929 - min_samples_leaf=1 n_estimator=1000 min_df=2 ngram_range(1,4)

app 0.22272884695198775 auc 0.6977777777777777 - min_samples_leaf=1 n_estimator=1000 min_df=3

ap 0.23694506856984365 auc 0.7427777777777778 - min_samples_leaf=2 n_estimator=1000 min_df=2

ap 0.21993029679545967 auc 0.7484313725490197 - min_samples_leaf=3 n_estimator=1000 min_df=2

 ap 0.27622068626834806 auc 0.7574836601307189 - min_samples_leaf=1 n_estimator=100 min_df=2

 ap 0.21318808251531807 auc 0.7309150326797386 - min_samples_leaf=2 n_estimator=100 min_df=2

## LightGBM

In [21]:
!pip install lightgbm

In [22]:
from lightgbm import LGBMClassifier

In [23]:
mdl = LGBMClassifier(random_state=0, class_weight='balanced', n_jobs=6)
mdl.fit(Xtrain_wtitle, y_train)

LGBMClassifier(class_weight='balanced', n_jobs=6, random_state=0)

In [24]:
p = mdl.predict_proba(Xval_wtitle)[:,1]

D:\Users\angel\Anaconda3\lib\site-packages\lightgbm\basic.py:739: UserWarning: Converting data to scipy sparse matrix.
  _log_warning('Converting data to scipy sparse matrix.')


In [25]:
average_precision_score(y_val, p)

0.13568143704270252

In [26]:
roc_auc_score(y_val, p)

0.663611111111111

## Bayesian Optimization

In [27]:
!pip install scikit-optimize

In [28]:
def tune_lgbm(params: dict):
    print(params)

    lr = params[0]
    max_depth = params[1]
    min_child_samples = params[2]
    subsample = params[3]
    colsample_bytree = params[4]
    n_estimator = params[5]
    min_df = params[6]
    ngram_range = (1, params[7])

    title_vec = TfidfVectorizer(min_df=min_df, ngram_range=ngram_range)
    title_bow_train = title_vec.fit_transform(title_train)
    title_bow_val = title_vec.transform(title_val)

    Xtrain_wtitle = hstack([X_train, title_bow_train])
    Xval_wtitle = hstack([X_val, title_bow_val])

    mdl = LGBMClassifier(learning_rate=lr, num_leaves=2**max_depth, max_depth=max_depth, min_child_samples=min_child_samples, subsample=subsample, colsample_bytree=colsample_bytree, bagging_freq=1, n_estimators=n_estimator, random_state=0, class_weight='balanced', n_jobs=6)

    mdl.fit(Xtrain_wtitle, y_train)

    p = mdl.predict_proba(Xval_wtitle)[:,1]
    print(roc_auc_score(y_val, p))

    return -average_precision_score(y_val, p)

In [29]:
from skopt import forest_minimize

space = [(1e-3, 1e-1, 'log-uniform'), #lr
         (1, 15), # max_depth
         (1, 20), # min_child_samples
         (0.05, 1.), # subsample
         (0.05, 1.), # colsample_bytree
         (100, 1000), # n_estimators
         (1,5), # min_df
         (1,5) # ngram_range
]

res = forest_minimize(tune_lgbm, space, random_state=160745, n_random_starts=20, n_calls=50, verbose=1)

77645206, 312, 4, 3]
[LightGBM] [Warning] bagging_freq is set=1, subsample_freq=0 will be ignored. Current value: bagging_freq=1
D:\Users\angel\Anaconda3\lib\site-packages\lightgbm\basic.py:739: UserWarning: Converting data to scipy sparse matrix.
  _log_warning('Converting data to scipy sparse matrix.')
0.666830065359477
Iteration No: 17 ended. Evaluation done at random point.
Time taken: 6.3013
Function value obtained: -0.1615
Current minimum: -0.2251
Iteration No: 18 started. Evaluating function at random point.
[0.004955229758078229, 5, 5, 0.06939551310802591, 0.4193273080472823, 725, 4, 1]
[LightGBM] [Warning] bagging_freq is set=1, subsample_freq=0 will be ignored. Current value: bagging_freq=1
D:\Users\angel\Anaconda3\lib\site-packages\lightgbm\basic.py:739: UserWarning: Converting data to scipy sparse matrix.
  _log_warning('Converting data to scipy sparse matrix.')
0.6517973856209149
Iteration No: 18 ended. Evaluation done at random point.
Time taken: 0.2613
Function value obt

In [30]:
res.x

[0.011144682437501488, 2, 5, 0.890697915175696, 0.06794603708438668, 246, 3, 4]

In [31]:
tune_lgbm(res.x)

[0.011144682437501488, 2, 5, 0.890697915175696, 0.06794603708438668, 246, 3, 4]
[LightGBM] [Warning] bagging_freq is set=1, subsample_freq=0 will be ignored. Current value: bagging_freq=1
0.7532679738562091
D:\Users\angel\Anaconda3\lib\site-packages\lightgbm\basic.py:739: UserWarning: Converting data to scipy sparse matrix.
  _log_warning('Converting data to scipy sparse matrix.')


-0.24897408670745122

ap 0.24897408670745122  auc 0.7532679738562091 - LGBM

ap 0.2701322613168307 auc 0.7720751633986929 - min_samples_leaf=1 n_estimator=1000 min_df=2 ngram_range(1,4) RF

## Logistic Regression

In [32]:
from sklearn.linear_model import LogisticRegression
from sklearn.preprocessing import MaxAbsScaler, StandardScaler
from scipy.sparse import csr_matrix

In [33]:
Xtrain_wtitle2 = csr_matrix(Xtrain_wtitle.copy())
Xval_wtitle2 = csr_matrix(Xval_wtitle.copy())

In [34]:
#scaler = StandardScaler()
scaler = MaxAbsScaler()

# Xtrain_wtitle2[:, :2] = scaler.fit_transform(Xtrain_wtitle2[:, :2].todense())
# Xval_wtitle2[:, :2] = scaler.transform(Xval_wtitle2[:, :2].todense())

Xtrain_wtitle2 = scaler.fit_transform(Xtrain_wtitle2)
Xval_wtitle2 = scaler.transform(Xval_wtitle2)

In [35]:
Xtrain_wtitle2.shape, Xval_wtitle2.shape

((645, 1550), (662, 1550))

In [36]:
mdl = LogisticRegression(C=10, n_jobs=6, random_state=0)
mdl.fit(Xtrain_wtitle2, y_train)

LogisticRegression(C=10, n_jobs=6, random_state=0)

In [37]:
p = mdl.predict_proba(Xval_wtitle2)[:, 1]

In [38]:
average_precision_score(y_val, p), roc_auc_score(y_val, p)

(0.26927648885823324, 0.7180718954248365)

(0.26533580463953477, 0.7552941176470589) - sem tunning - standarscaler

(0.25495170554301794, 0.7511111111111111) - sem tunning - maxabsscaler

(0.26927648885823324, 0.7180718954248365) - C=10 - maxabsscaler

(0.24767994545053074, 0.7578431372549019) - C=0.5 - maxabsscaler